In [ ]:
import numpy as np
from PIL import Image
import cv2
import time
import pyautogui
import keyboard

import torch
from torchvision import transforms
from torchvision import models


width = 480
height = 270

    
def forward():
    keyboard.press('w')
    time.sleep(0.04)
    keyboard.release('w')

def left():
    keyboard.press('a')
    keyboard.press('w')
    time.sleep(0.04)
    keyboard.release('a')
    keyboard.release('w')

def right():
    keyboard.press('d')
    keyboard.press('w')
    time.sleep(0.04)
    keyboard.release('d')
    keyboard.release('w')
    
def brake():
    keyboard.press('s')
    time.sleep(0.04)
    keyboard.release('s')


# # Load trained model
model = models.vgg16(weights=None)
model.classifier[-1] = torch.nn.Linear(in_features=4096, out_features=4)
model.load_state_dict(torch.load('models/modelname.type', map_location=torch.device('cuda')))
model.eval()


for i in range(5, 0, -1):       
        print(f"Time left: {i} seconds", end='\r')
        time.sleep(1) 

# Pause to change to GTA windows
time.sleep(4)

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((480, 270)),
    transforms.ToTensor(),
])

# Initialize the camera
cap = cv2.VideoCapture(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("running VGG auto-pilot...................")

def main():
    
    while True:

        ret, frame = cap.read()

        if not ret:
            break

        frame = cv2.resize(frame, (width, height))
        cv2.imshow('camera', frame)
        screen_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        screen_img = transform(screen_img).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(screen_img)
            _, predicted = torch.max(output, 1)
            prediction = predicted.item()
            
            #print probas
            # print("Probass:", output)
        
        if prediction == 0:
            brake()
            print(f"{output} brake")
        elif prediction == 1:
            forward()
            print(f"{output} forward")
        elif prediction == 2:
            left()
            print(f"{output} forward + left")
        elif prediction == 3:
            right()
            print(f"{output} forward + right")
        

        if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
            
if __name__ == '__main__':            
    main()

# Release the camera
# cap.release()
